In [16]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
# nltk.download('stopwords');
from nltk.stem import PorterStemmer
stemming = PorterStemmer()
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))                  
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import yellowbrick
from yellowbrick.text import FreqDistVisualizer
from yellowbrick.text import TSNEVisualizer
from sklearn.feature_extraction import text 

In [17]:
# import the dataset
def get_data(file1,file2, file3, path = 'C:/Gradient Boost/')->pd.DataFrame:
    """
    gets input csv file and reads it into a pandas dataframe
    
    returns pandas dataframe
    
    """
    df1=pd.read_csv(path + file1,encoding='latin-1')
    df1['State'] = 'Abuja'
    df2=pd.read_csv(path + file2,encoding='latin-1')
    df2['State'] = 'Lagos'
    df3=pd.read_csv(path + file3,encoding='latin-1')
    df3['State'] = 'Kaduna'
    df=pd.concat([df1, df2, df3])
    return df


In [18]:
def explore_data(df):
    """
    Explore input data frame.
    
    Returns None
    """
    print(df.shape) # Shape of the data frame
    print(df.columns) # Columns in the data frame
    print(f"number of unique labels, {df['Rating'].nunique()}") # number of labels

In [19]:
#  word_tokenize to all records, making a new column in our DataFrame strip out non alphanumeric words/characters (such as numbers and punctuation) using .isalpha.
def identify_tokens(df):
    review = df['Review_Text']
    tokens = nltk.word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

In [20]:
# Stemming reduces related words to a common stem. It is an optional process step, and it is useful to test accuracy with and without stemming.
def stem_list(df):
    my_list = df['words']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

In [21]:
# Removing stop words
def remove_stops(df):
    my_list = df['words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

In [22]:
# Rejoin words
def rejoin_words(df):
    my_list = df['stem_meaningful']
    joined_words = ( " ".join(my_list))
    return joined_words

In [23]:
# Using TextBlob for Polarity
def preprocess(ReviewText):
    ReviewText = ReviewText.str.replace("(<br/>)", "")
    ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
    ReviewText = ReviewText.str.replace('(&amp)', '')
    ReviewText = ReviewText.str.replace('(&gt)', '')
    ReviewText = ReviewText.str.replace('(&lt)', '')
    ReviewText = ReviewText.str.replace('(\xa0)', ' ')  
    return ReviewText


In [24]:
df = get_data('/Abuja Hotels Cleaned.txt', '/Lagos Hotels Cleaned.txt', '/Kaduna Hotels Cleaned.txt')
df

,Unnamed: 0,Hotel_Title,Title_URL,Address,Address_URL,City,Review_Text,Price,Rating,State
0,0,Grand Ibro Hotel,https://hotels.ng/hotel/34353-grand-ibro-hotel...,"Wuse, Abuja - 2121 Micheal Okpara Road Opposit...",https://hotels.ng/hotels-in-abuja/wuse,Wuse,"Good stay, my stay was very fine....",13000.0,7.4,Abuja
1,1,Power Mike Hotels,https://hotels.ng/hotel/995346-power-mike-hote...,"Garki, Abuja - 9 Argungu Close , Off Benue Cre...",https://hotels.ng/hotels-in-abuja/garki,Garki,The room was reasonably tidy with a very big b...,5000.0,7.3,Abuja
2,2,New Rendezvous Hotel,https://hotels.ng/hotel/36783-new-rendezvous-h...,"Wuse 6, Abuja - 21, Plot 188, Makeni Street, Z...",https://hotels.ng/hotels-in-abuja/wuse-6,Wuse 6,"Very clean room, ventilation was not too good,...",5000.0,6.9,Abuja
3,3,Summit Villas Suites,https://hotels.ng/hotel/37489-summit-villas-su...,"Gwarinpa, Abuja - Life Camp Junction, Opposit...",https://hotels.ng/hotels-in-abuja/gwarinpa,Gwarinpa,The hotel is beautiful and fully equipped. Hos...,8400.0,6.8,Abuja
4,4,Orient Hotel,https://hotels.ng/hotel/32651-orient-hotel-abuja,"Wuse, Abuja - 12 Sudan Street",https://hotels.ng/hotels-in-abuja/wuse,Wuse,My stay was okay at the hotel. There were no i...,9000.0,7.4,Abuja
...,...,...,...,...,...,...,...,...,...,...
20,61,Silversand Hotel,https://hotels.ng/hotel/85725-silversand-hotel...,"Ungwar Sarki, Kaduna - 19,Katuru Road,Off Isa ...",https://hotels.ng/hotels-in-kaduna/ungwar-sarki,Ungwar Sarki,The food at the hotel was very nice and afford...,13680.0,6.8,Kaduna
21,79,Kabir Gymnasium,https://hotels.ng/hotel/98733-kabir-gymnasium-...,"Kaduna, Kaduna - 12 Abdulrahman Okene Road, Op...",https://hotels.ng/hotels-in-kaduna/kaduna,Kaduna,I made use of the hotel while on a personal tr...,7000.0,6.8,Kaduna
22,83,Maharaja Hotel,https://hotels.ng/hotel/25937-maharaja-hotel-k...,"Zaria, Kaduna - 1, Independence Way, Kaduna Ni...",https://hotels.ng/hotels-in-kaduna/zaria,Zaria,The hotel is a good place to be because I enjo...,9600.0,6.8,Kaduna
23,116,Excel Inn,https://hotels.ng/hotel/43881-excel-inn-akwa-ibom,"Kaduna North, Kaduna - NN23 Constitution Road",https://hotels.ng/hotels-in-kaduna/kaduna-north,Kaduna North,\n\nI enjoyed my stay at the hotel. It is very...,5610.0,6.8,Kaduna


 >  **Data Preparation** 

In [25]:
# convert all text to lower case.
df['Review_Text'] = df['Review_Text'].str.lower()

# identify tokens
df['words'] = df.apply(identify_tokens, axis=1)

# stem words
df['stemmed_words'] = df.apply(stem_list, axis=1)

# Remove stop words
df['stem_meaningful'] = df.apply(remove_stops, axis=1)

# rejoin words
df['processed'] = df.apply(rejoin_words, axis=1)

# remove nan rows
df = df[~df['Review_Text'].isnull()]




 >  **Sentiment Polarity** 



In [26]:
# Rendering Polarity
df['Review_Text'] = preprocess(df['Review_Text'])
df['polarity'] = df['Review_Text'].map(lambda text: TextBlob(text).sentiment.polarity)

<ipython-input-23-458fc61ecad2>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace("(<br/>)", "")
<ipython-input-23-458fc61ecad2>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
<ipython-input-23-458fc61ecad2>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(&amp)', '')
<ipython-input-23-458fc61ecad2>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(&gt)', '')
<ipython-input-23-458fc61ecad2>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(&lt)', '')
<ipython-input-23-458fc61ecad2>:8: FutureWarning: The default value of regex will cha

In [27]:
df

,Unnamed: 0,Hotel_Title,Title_URL,Address,Address_URL,City,Review_Text,Price,Rating,State,words,stemmed_words,stem_meaningful,processed,polarity
0,0,Grand Ibro Hotel,https://hotels.ng/hotel/34353-grand-ibro-hotel...,"Wuse, Abuja - 2121 Micheal Okpara Road Opposit...",https://hotels.ng/hotels-in-abuja/wuse,Wuse,"good stay, my stay was very fine....",13000.0,7.4,Abuja,"[good, stay, my, stay, was, very, fine]","[good, stay, my, stay, wa, veri, fine]","[good, stay, stay, fine]",good stay stay fine,0.620833
1,1,Power Mike Hotels,https://hotels.ng/hotel/995346-power-mike-hote...,"Garki, Abuja - 9 Argungu Close , Off Benue Cre...",https://hotels.ng/hotels-in-abuja/garki,Garki,the room was reasonably tidy with a very big b...,5000.0,7.3,Abuja,"[the, room, was, reasonably, tidy, with, a, ve...","[the, room, wa, reason, tidi, with, a, veri, b...","[room, reasonably, tidy, big, bed, slept, comf...",room reasonably tidy big bed slept comfortably...,0.333333
2,2,New Rendezvous Hotel,https://hotels.ng/hotel/36783-new-rendezvous-h...,"Wuse 6, Abuja - 21, Plot 188, Makeni Street, Z...",https://hotels.ng/hotels-in-abuja/wuse-6,Wuse 6,"very clean room, ventilation was not too good,...",5000.0,6.9,Abuja,"[very, clean, room, ventilation, was, not, too...","[veri, clean, room, ventil, wa, not, too, good...","[clean, room, ventilation, good, facilities, c...",clean room ventilation good facilities conveni...,0.592222
3,3,Summit Villas Suites,https://hotels.ng/hotel/37489-summit-villas-su...,"Gwarinpa, Abuja - Life Camp Junction, Opposit...",https://hotels.ng/hotels-in-abuja/gwarinpa,Gwarinpa,the hotel is beautiful and fully equipped. hos...,8400.0,6.8,Abuja,"[the, hotel, is, beautiful, and, fully, equipp...","[the, hotel, is, beauti, and, fulli, equip, ho...","[hotel, beautiful, fully, equipped, hospitable...",hotel beautiful fully equipped hospitable resp...,0.850000
4,4,Orient Hotel,https://hotels.ng/hotel/32651-orient-hotel-abuja,"Wuse, Abuja - 12 Sudan Street",https://hotels.ng/hotels-in-abuja/wuse,Wuse,my stay was okay at the hotel. there were no i...,9000.0,7.4,Abuja,"[my, stay, was, okay, at, the, hotel, there, w...","[my, stay, wa, okay, at, the, hotel, there, we...","[stay, okay, hotel, issues, throughout, stay, ...",stay okay hotel issues throughout stay well re...,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,61,Silversand Hotel,https://hotels.ng/hotel/85725-silversand-hotel...,"Ungwar Sarki, Kaduna - 19,Katuru Road,Off Isa ...",https://hotels.ng/hotels-in-kaduna/ungwar-sarki,Ungwar Sarki,the food at the hotel was very nice and afford...,13680.0,6.8,Kaduna,"[the, food, at, the, hotel, was, very, nice, a...","[the, food, at, the, hotel, wa, veri, nice, an...","[food, hotel, nice, affordable, restaurant, ni...",food hotel nice affordable restaurant nice table,0.690000
21,79,Kabir Gymnasium,https://hotels.ng/hotel/98733-kabir-gymnasium-...,"Kaduna, Kaduna - 12 Abdulrahman Okene Road, Op...",https://hotels.ng/hotels-in-kaduna/kaduna,Kaduna,i made use of the hotel while on a personal tr...,7000.0,6.8,Kaduna,"[i, made, use, of, the, hotel, while, on, a, p...","[i, made, use, of, the, hotel, while, on, a, p...","[made, use, hotel, personal, trip, kaduna, kad...",made use hotel personal trip kaduna kaduna p,0.000000
22,83,Maharaja Hotel,https://hotels.ng/hotel/25937-maharaja-hotel-k...,"Zaria, Kaduna - 1, Independence Way, Kaduna Ni...",https://hotels.ng/hotels-in-kaduna/zaria,Zaria,the hotel is a good place to be because i enjo...,9600.0,6.8,Kaduna,"[the, hotel, is, a, good, place, to, be, becau...","[the, hotel, is, a, good, place, to, be, becau...","[hotel, good, place, enjoyed, stay, facilities...",hotel good place enjoyed stay facilities services,0.600000
23,116,Excel Inn,https://hotels.ng/hotel/43881-excel-inn-akwa-ibom,"Kaduna North, Kaduna - NN23 Constitution Road",https://hotels.ng/hotels-in-kaduna/kaduna-north,Kaduna North,\n\ni enjoyed my stay at the hotel. it is very...,5610.0,6.8,Kaduna,"[i, enjoyed, my